In [8]:
import pandas as pd

df = pd.read_csv('input_test.csv', encoding='latin1', header=None, names=["text"])
df.head()


,text
0,Long Story short: This game is absolutely wond...
1,Don't be daunted by the price (granted it's su...
2,It's hard to describe the awesomeness of this ...
3,This game is amazing!!!<br /><br />I've been p...
4,Its a solid attempt to turn Dungeons & Dragons...


### Remove punctuation/lower casing

In [9]:
# Load the regular expression library
import re
# Remove punctuation
df['text'] = df['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the to lowercase
df['text'] = df['text'].map(lambda x: x.lower())

### Tokenize

In [10]:
%%time
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df["text"].values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['long', 'story', 'short', 'this', 'game', 'is', 'absolutely', 'wonderful', 'this', 'according', 'to', 'many', 'board', 'game', 'review', 'sites', 'is', 'set', 'to', 'be', 'the', 'top', 'game', 'of', 'and', 'recommend', 'getting', 'yourself', 'copy', 'though', 'it', 'will', 'likely', 'be', 'difficult', 'take', 'some', 'time', 'its', 'definitely', 'hot', 'selling', 'item', 'and', 'was', 'lucky', 'enough', 'to', 'pre', 'order', 'mine', 'month', 'ago', 'before', 'other', 'folks', 'started', 'seeing', 'it', 'listed', 'on', 'amazon', 'this', 'is', 'definitely', 'worth', 'getting', 'and', 'will', 'make', 'huge', 'hit', 'to', 'your', 'gaming', 'nights', 'more', 'detailed', 'breakdown', 'is', 'below', 'br', 'note', 'relating', 'to', 'pictures', 'the', 'game', 'does', 'not', 'come', 'with', 'painted', 'miniatures', 'as', 'enjoy', 'paining', 'them', 'did', 'so', 'but', 'my', 'wife', 'and', 'played', 'several', 'games', 'before', 'they', 'were', 'painted', 'without', 'any', 'issues', 'confusion'

### Bi-grams and Tri-grams

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Remove stopwords, make bigrams and trigrams, and lemmatization functions definitions

In [15]:
# Uncomment two lines below to download nltk data for first time, after download, comment again

#import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /home/saber/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

### Building the LDA model

In [ ]:
# You can change num_topics
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]